In [1]:
import time
import pandas as pd
import os

from pytrends.request import TrendReq
from datetime import datetime, timedelta

In [2]:
# read data
input_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/data/final_data'

os.chdir(input_path)
data = pd.read_csv('filtered_data_w_ratings.csv')

In [3]:
pytrends = TrendReq(hl='en-US', tz=360)

In [4]:
def fetch_trend(movie_title, release_date):
    # calculate the start date
    start_date = release_date - timedelta(days=365)

    # change the datetime format
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = release_date.strftime('%Y-%m-%d')

    # build the payload
    pytrends.build_payload(kw_list=[movie_title], timeframe=f'{start_date_str} {end_date_str}')

    # fetch the interest over time
    trends = pytrends.interest_over_time()

    # return the mean (we can change mean to other statistics)
    return trends[movie_title].mean()

In [12]:
data_sample = data[1000:1100]

for index, row in data_sample.iterrows():
    try: 
        # convert to datetime object
        release_date = pd.to_datetime(row['release_date'])

        # retrieve the data
        trend_index = fetch_trend(row['primaryTitle'], release_date)

        # store the data
        data_sample.at[index, 'search_index'] = trend_index

        # to avoid the error that reach the maximum retrieve matrix
        time.sleep(15)
    except Exception as e:
        print(f'error: {e}')
        data_sample.at[index, 'error_message'] = e
        time.sleep(15)
        continue

<ipython-input-12-05e7e9ff2ac9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample.at[index, 'search_index'] = trend_index


error: The request failed: Google returned a response with code 500


<ipython-input-12-05e7e9ff2ac9>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample.at[index, 'error_message'] = e


error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22Meek%27s+Cutoff%22%2C+%22time%22%3A+%222010-04-08+2011-04-08%22%2C+%22geo%22%3A+%22%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1123)')))
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/widgetdata/multiline?req=%7B%22time%22%3A+%222001-01-25+2002-01-25%22%2C+%22resolution%22%3A+%22WEEK%22%2C+%22locale%22%3A+%22

In [13]:
output_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/searching_index/results'
os.chdir(output_path)
data_sample.to_csv('results_index1000_1100.csv')